### Import required libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import warnings
from hmmlearn import hmm


### Suppress warnings and set plot style

In [ ]:
warnings.simplefilter(action ='ignore')
plt.style.use('seaborn')

### Read from the CSV File

In [ ]:
df = pd.read_csv('weatherAUS.csv')
df.head()

### Look at the statistics of the dataset

In [ ]:
df.describe()

### Look at number of Nulls and data types

In [ ]:
data_types = pd.DataFrame(df.dtypes, columns=['Data Types'])
count_of_nulls = pd.DataFrame(df.isnull().sum(), columns=['Count of Nulls'])
data_dictionary = pd.concat([data_types, count_of_nulls], axis='columns')
data_dictionary

### Remove Columns having high percentage of NaNs

In [ ]:
col = ['Sunshine', 'Evaporation', 'Cloud9am', 'Cloud3pm']
df.drop(col, axis=1, inplace=True)
df

### Replace the NaNs by mean in (Int, Float) and last value in (Date, Location)

In [ ]:
# Replace numerical columns with mean
def replace_numerical(df1):
    for col in df1.select_dtypes(['int', 'float']):
        df1[col] = df1[col].fillna(df1[col].mean())
    return df1

# Replace object columns with last row value 
def replace_object(df1):
    for col in df1.select_dtypes('object'):
        df1[col] = df1[col].fillna(method='ffill')
    return df1

In [ ]:
df = replace_numerical(df)
df= replace_object(df)

### Check no NaNs values

In [ ]:
df.isnull().sum()

### Visualize the rainfall in each year 

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
rainfall =[df['Date'].dt.year, df['Date'].dt.month, df['Rainfall']]
headers = ['Year', 'Month', 'Rainfall']
rainfall_df = pd.concat(rainfall, axis=1, keys=headers)

plt.figure(figsize=(10,8))
a = rainfall_df.groupby('Month').agg({'Rainfall':'sum'})
a.plot(kind='bar', color='green')
plt.title('Rainfall distribution in each month', fontsize=15)
plt.xlabel('Month', fontsize=10)
plt.ylabel('Rainfall (in mm)', fontsize=10)
plt.xticks(rotation=0)

### Visualize the rainfall across all different regions of Australia

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(df['Location'],df['Rainfall'])
plt.xlabel("Location")
plt.xticks(rotation=90)
plt.ylabel("Rainfall")
plt.show()

In [ ]:
df["Date"].dtype

### Visualize the distributions of the features

In [ ]:
r=4
eps=3
continuous = [col for col in df.columns if (df[col].dtype != object and df[col].dtype !=df["Date"].dtype) ]

fig, axes = plt.subplots(nrows=r,ncols=int(len(continuous)/r), figsize=(12, 18))
axes = axes.reshape(-1)

for i, col in enumerate(continuous):
    sns.histplot(df[col], ax=axes[i])
    left=min(df[col])+eps
    right=max(df[col])+eps
    ##print(left,right)
    if col=="Rainfall":
      axes[i].set_xlim(0, 1)
   

In [ ]:
continuous

In [ ]:
df.dtypes

In [ ]:
# comparison of WindGustDir, WindDir9am and WindDir3pm

fig, ax = plt.subplots(3, 1, figsize=(10,8))

# WindGustDir
sns.countplot(x=df['WindGustDir'], palette=['#9b59b6','#3498db'], ax=ax[0])
ax[0].set_title("Wind Gust Direction", fontsize=10)

# WindGustDir
sns.countplot(x=df['WindDir9am'], palette= ['#7fcdbb' , '#fc9272'], ax=ax[1])
ax[1].set_title("Wind Direction at 9AM", fontsize=10)

# WindGustDir
sns.countplot(x=df['WindDir3pm'], palette=['#432371',"#FAAE7B"], ax=ax[2])
ax[2].set_title("Wind Direction at 3PM", fontsize=10)
fig.tight_layout()

In [ ]:
cat=['WindGustDir','WindDir9am','WindDir3pm']

In [ ]:
df.drop(cat,axis=1,inplace=True) 

In [ ]:
df.columns

### Convert the categorical values to one-hot encoded vectors

In [ ]:
#one_hot_encoded_df = pd.get_dummies(df, columns = ['WindGustDir','WindDir9am','WindDir3pm'])
#print((one_hot_encoded_df).columns)

In [ ]:
from sklearn.preprocessing import LabelEncoder
Label_Encoder = LabelEncoder()
df['RainToday']=Label_Encoder.fit_transform(df['RainToday'].astype(str))
df['RainTomorrow']=Label_Encoder.fit_transform(df['RainTomorrow'].astype(str))


In [ ]:
#one_hot_encoded_df.head()

### Get Subset of all the data for Cairns(which has max rainfall)

In [ ]:
df_cairns=df[df['Location']=='Cairns']

In [ ]:
df_cairns.head()

### Split into target variable and input

In [ ]:
df_cairns_x = df_cairns.drop(['RainTomorrow','RainToday','Location','Date'],axis=1)

In [ ]:
df_cairns_x.head()

In [ ]:
df_cairns_norm=df_cairns_x.apply(lambda x: (x-x.mean())/ x.std(), axis=0)

In [ ]:
df_cairns_norm.head()

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(15,25))

# WindSpeed9am
sns.distplot(df_cairns_norm['WindSpeed9am'], ax=ax[0,0], color='green')
ax[0,0].set_title("Wind Speed at 9AM", fontsize=15)

# WindSpeed3pm
sns.distplot(df_cairns_norm['WindSpeed3pm'], ax=ax[0,1], color='green')
ax[0,1].set_title("Wind Speed at 3PM", fontsize=15)

# Humidity9am
sns.distplot(df_cairns_norm['Humidity9am'], ax=ax[1,0], color='orange')
ax[1,0].set_title("Humidity at 9AM", fontsize=15)

# Humidity3pm
sns.distplot(df_cairns_norm['Humidity3pm'], ax=ax[1,1], color='orange')
ax[1,1].set_title("Humidity at 3PM", fontsize=15)

# Pressure9am
sns.distplot(df_cairns_norm['Pressure9am'], ax=ax[2,0], color='red')
ax[2,0].set_title("Pressure at 9AM", fontsize=15)

# Pressure3pm
sns.distplot(df_cairns_norm['Pressure3pm'], ax=ax[2,1], color='red')
ax[2,1].set_title("Pressure at 3PM", fontsize=15)

# Temp9am
sns.distplot(df_cairns_norm['Temp9am'], ax=ax[3,0], color='blue')
ax[3,0].set_title("Temperature at 9AM", fontsize=15)

# Temp3pm
sns.distplot(df_cairns_norm['Temp3pm'], ax=ax[3,1], color='blue')
ax[3,1].set_title("Temperature at 3PM", fontsize=15)


In [ ]:
x = df_cairns_norm.values
y = df_cairns['RainTomorrow'].values

In [ ]:
print(type(x),type(y))

In [ ]:
print(x[0])
print(y[0:5])

In [ ]:
x.shape

### Split into training and test set

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.01,random_state=100) 

In [ ]:
assert np.count_nonzero(np.isfinite(x_train))==(x_train.shape[0])*(x_train.shape[1])
assert np.count_nonzero(np.isfinite(x_test))==(x_test.shape[0])*(x_test.shape[1])

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
## Define the parameters of the HMM 

num_states = 30
emission_dim = 12 

#num_lags = 1
##sgd_hmm = LinearAutoregressiveHMM(num_states=num_states, emission_dim=emission_dim, num_lags=num_lags,transition_matrix_stickiness=10.0)
##train_emissions =np.copy(np.around(x_train,decimals=2))


### Visualize the clustering of the training set

In [ ]:
### Plot the elbow plot to find optimal value of n_clusters

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

distortions = []
inertias = []
mapping1 = {}
mapping2 = {}

K = range(1, 10)
  
for k in K:
    
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(x_train)
    kmeanModel.fit(x_train)
  
    distortions.append(sum(np.min(cdist(x_train, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / x_train.shape[0])
    inertias.append(kmeanModel.inertia_)
  
    mapping1[k] = sum(np.min(cdist(x_train, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / x_train.shape[0]
    mapping2[k] = kmeanModel.inertia_
    
plt.plot(K, distortions, 'bx-')
plt.xlabel('Values of K')
plt.ylabel('Distortions')
plt.title('The Elbow Method using Distortion')
plt.show()    

In [ ]:
### Use k-means to categorize the 12 continuous features into clusters

In [ ]:
kmeans = KMeans(n_clusters=num_states, random_state=0, n_init="auto").fit(x_train)
print(kmeans.labels_)
print(kmeans.cluster_centers_)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(2)    # PCA object for 2-D plot

#Transform the data
pca_train = pca.fit_transform(x_train)
pca_test = pca.transform(x_test)


print("Explained Variance",pca.explained_variance_ratio_)
print("Pca transformed Data", pca_train)



In [ ]:
print(pca_test)

In [ ]:
color = [0,1,2,3,4,5,6]

In [ ]:
#Getting unique labels
u_labels = np.unique(kmeans.labels_)
train_label = kmeans.labels_
test_label = kmeans.predict(x_test)

#plotting the results:
for i in u_labels:
    plt.scatter(pca_train[train_label == i , 0] , pca_train[train_label == i , 1] , label=i)
    ##plt.scatter(pca_test[test_label == i , 0] , pca_test[test_label == i , 1] , label=color[i], marker="v")
    
plt.title("The Visualization of Clusters")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
##plt.legend()
plt.show()

In [ ]:
### Use the train_label as hidden states and the binary array y_train as the emissions

In [ ]:
## Implement the HMM Model by giving input (hiddenstate, emission) pairs

In [ ]:
# Define the set of hidden states and emissions
hidden_states =list(range(0,num_states))
emissions = [0, 1]

# Define the observed sequence and corresponding hidden state sequence
observed_sequence = (y_train).tolist()
hidden_state_sequence = train_label.tolist()

print("Observed Sequence Sample",observed_sequence[0:10])
print("Hidden State Sample",hidden_state_sequence[0:10])

# Compute the emission matrix
emission_matrix = np.zeros((len(hidden_states), len(emissions)))

for i, state in enumerate(hidden_states):
    for j, emission in enumerate(emissions):
        count = sum(1 for k in range(len(observed_sequence)) if observed_sequence[k] == emission and hidden_state_sequence[k] == state)
        emission_matrix[i][j] = count / sum(1 for k in range(len(hidden_state_sequence)) if hidden_state_sequence[k] == state)

# Compute the transition matrix
transition_matrix = np.zeros((len(hidden_states), len(hidden_states)))
for i, state1 in enumerate(hidden_states):
    for j, state2 in enumerate(hidden_states):
        count = sum(1 for k in range(len(hidden_state_sequence)-1) if hidden_state_sequence[k] == state1 and hidden_state_sequence[k+1] == state2)
        transition_matrix[i][j] = count / sum(1 for k in range(len(hidden_state_sequence)-1) if hidden_state_sequence[k] == state1)

# Print the computed matrices
print("Emission matrix:")
print(emission_matrix)
print("Transition matrix:")
print(transition_matrix)

In [ ]:
test_label

In [ ]:
# Predict on the test set (using the hidden states, predict observations 0/1)

In [ ]:
## Sample using the emission probabilities
import random
test_predy=[]

for i in test_label:
   
   #if (emission_matrix[i][0]>=emission_matrix[i][1])
    #   test_predy.append(0);
   #else:
    #   test_predy.append(1);

   r = random.uniform(0, 1)
   
   if (r<emission_matrix[i][0]):
        test_predy.append(0);
        
   else:
        test_predy.append(1);
        
      

In [ ]:
test_predy

In [ ]:
y_test

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_predy,y_test)

In [ ]:
accuracy*100